In [1]:
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import pandas as pd
import numpy as np
import os
import re

In [2]:
fields = ['episode', 'reward', 'ep_block_prob', 'ep_accepted_requests', 'avg_deployment_cost',
        'avg_total_latency', 'avg_access_latency', 'avg_proc_latency',
        'avg_throuput_in', 'avg_packetsize_in', 'avg_interarrival_in', 
        'avg_throuput_out','avg_packetsize_out','avg_interarrival_out',
        'avg_qoe','gini', 'execution_time']

In [3]:
def metric_mean(x):
    return np.mean(x)
#for col in ['test_avg_qoe','test_gini','test_avg_total_latency','test_avg_deployment_cost']:
#    df[col] = df[col].apply(metric_mean)

## Copare mean during training and testing

In [4]:
import os
import re
import pandas as pd

# Directory paths for training and testing logs
train_dir = './run_metrics'
test_dir = './run_metrics_test'

# Metric columns to calculate means
metric_columns = ['avg_qoe', 'gini', 'avg_total_latency', 'avg_deployment_cost', 'ep_accepted_requests']

# Listing training and testing files
train_files = set(sorted([f for f in os.listdir(train_dir) if f.endswith('.csv')]))
test_files = set(sorted([f for f in os.listdir(test_dir) if f.endswith('.csv')]))
names = list(train_files.intersection(test_files))

# Pattern to extract variables from the filename
pattern = r"(?P<alg>.+)_env_(?P<env_name>.+)_qoe_(?P<qoe_included>.+)_obj_(?P<obj_included>.+)_cw_(?P<cost_weight>.+)_qw_(?P<qoe_weight>.+)_lw_(?P<latency_weight>.+)_gw_(?P<gini_weight>.+)_sim_(?P<qoe_simulation_mode>.+)_acc_(?P<qoe_accuracy>.+)"

# List to hold all results
results = []

In [5]:
for file_name in names:
    # Match the pattern against the file name
    name = file_name.replace('.csv', '')
    match = re.match(pattern, name)
    
    if match:
        # Extract variables
        variables = match.groupdict()
        train_file = os.path.join(train_dir, file_name)
        test_file = os.path.join(test_dir, file_name)
        
        # Load training and testing data
        df_train = pd.read_csv(train_file, on_bad_lines='skip')
        df_test = pd.read_csv(test_file, on_bad_lines='skip')
    
        # Calculate means for metrics
        train_means = df_train[metric_columns]
        test_means = df_test[metric_columns]

        # Convert means to dictionaries and add prefixes to keys
        train_dic = {f"train_{k}": np.asanyarray(list(v.values()))[:100] for k, v in train_means.to_dict().items()}
        test_dic = {f"test_{k}": np.asanyarray(list(v.values()))[:100] for k, v in test_means.to_dict().items()}
        
        # Combine variables, train, and test dictionaries
        #combined_result = {**variables, **train_dic, **test_dic}
        combined_result = {**variables, **train_dic, **test_dic}
        

        # Add the combined result to the results list
        results.append(combined_result)

# Convert the list of results to a DataFrame
results_df = pd.DataFrame(results)


results_df = results_df.astype({'cost_weight': 'float64', 'qoe_weight': 'float64', 'latency_weight': 'float64', 'gini_weight': 'float64'})

In [6]:
results_df.shape

(132, 20)

# Analysing The Results

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
# Focus on columns of interest for analysis
columns_of_interest = [
    'cost_weight', 'qoe_weight', 'qoe_included',
    'train_avg_qoe', 'test_avg_qoe',
    'train_avg_deployment_cost', 'test_avg_deployment_cost',
]

In [9]:
df = results_df.copy()

In [10]:
df['obj_included'] = df['obj_included'].map({'True': True, 'False': False})
df['qoe_included'] = df['qoe_included'].map({'True': True, 'False': False})
#df['obj_included'] = df['obj_included'].astype(bool)
#df['qoe_included'] = df['obj_included'].astype(bool)
#df['obj_included'] = df['obj_included'].map({'True': True, 'False': False})
#df['qoe_included'] = df['qoe_included'].map({'True': True, 'False': False})

### Observation Space and Weight Strategy

In [11]:
import pandas as pd

# weight configurations
strategy_weights = {
    "Cost":        {'cost_weight': 1.0, 'latency_weight': 0.0, 'gini_weight': 0.0, 'qoe_weight': 0.0},
    "Latency":     {'cost_weight': 0.0, 'latency_weight': 1.0, 'gini_weight': 0.0, 'qoe_weight': 0.0},
    "Inequality":  {'cost_weight': 0.0, 'latency_weight': 0.0, 'gini_weight': 1.0, 'qoe_weight': 0.0},
    "Qoe":         {'cost_weight': 0.0, 'latency_weight': 0.0, 'gini_weight': 0.0, 'qoe_weight': 1.0},
    "CostLat":     {'cost_weight': 0.5, 'latency_weight': 0.5, 'gini_weight': 0.0, 'qoe_weight': 0.0},
    "CostIneq":    {'cost_weight': 0.5, 'latency_weight': 0.0, 'gini_weight': 0.5, 'qoe_weight': 0.0},
    "CostQoe":     {'cost_weight': 0.5, 'latency_weight': 0.0, 'gini_weight': 0.0, 'qoe_weight': 0.5},
    "LatIneq":     {'cost_weight': 0.0, 'latency_weight': 0.5, 'gini_weight': 0.5, 'qoe_weight': 0.0},
    "LatQoe":      {'cost_weight': 0.0, 'latency_weight': 0.5, 'gini_weight': 0.0, 'qoe_weight': 0.5},
    "QoeIneq":     {'cost_weight': 0.0, 'latency_weight': 0.0, 'gini_weight': 0.5, 'qoe_weight': 0.5},
    "Balanced":    {'cost_weight': 0.25, 'latency_weight': 0.25, 'gini_weight': 0.25, 'qoe_weight': 0.25},
}

# observation configurations
obs_configs = {
    'FF': {'obj_included': False, 'qoe_included': False},
    'FT': {'obj_included': True,  'qoe_included': False},
    'TF': {'obj_included': False, 'qoe_included': True},
    'TT': {'obj_included': True,  'qoe_included': True}
}

# Initialize result list
results = []

# Loop through each strategy and observation config
for strat_name, weights in strategy_weights.items():
    strat_df = df[
        (df['cost_weight'] == weights['cost_weight']) &
        (df['latency_weight'] == weights['latency_weight']) &
        (df['gini_weight'] == weights['gini_weight']) &
        (df['qoe_weight'] == weights['qoe_weight'])
    ]
    
    for config_name, filters in obs_configs.items():
        if (config_name != 'FF') and (config_name != 'FT'):           
            config_df = strat_df[
                (strat_df['obj_included'] == filters['obj_included']) &
                (strat_df['qoe_included'] == filters['qoe_included'])
            ]
            
        else:
            config_df = strat_df[
                (strat_df['obj_included'] == filters['obj_included']) &
                (strat_df['qoe_included'] == filters['qoe_included'])
            ]
            
        results.append({
            'Strategy': strat_name,
            'Obs_Config': config_name,
            'Train_Avg_QoE_Mean': np.mean(config_df['train_avg_qoe'].values[0]),
            'Test_Avg_QoE_Mean': np.mean(config_df['test_avg_qoe'].values[0]),
            
        })

# Convert to DataFrame
summary_df = pd.DataFrame(results)

# Display

In [12]:
summary_df

,Strategy,Obs_Config,Train_Avg_QoE_Mean,Test_Avg_QoE_Mean
0,Cost,FF,6.1594,5.9164
1,Cost,FT,5.9381,6.0617
2,Cost,TF,5.9653,6.1304
3,Cost,TT,6.0873,5.7764
4,Latency,FF,5.8662,6.1944
5,Latency,FT,6.0202,6.0560
6,Latency,TF,6.0585,6.1110
7,Latency,TT,6.0334,5.8774
8,Inequality,FF,6.2105,6.2565
9,Inequality,FT,6.0146,5.8902
